In [1]:
# Import需要的套件
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import torch.nn as nn
import torch
import torchvision.transforms as transforms
from collections import OrderedDict
import pandas as pd
import numpy as np
import time
import cv2
import os

%run Dataset.ipynb
%run Model.ipynb

## Raed date

In [2]:
#分別將 training set、validation set、testing set 用 readfile 函式讀進來
data_dir = '../Data/food-11'
print("Reading data")

train_x, train_y = readfile(os.path.join(data_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))

val_x, val_y = readfile(os.path.join(data_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))

test_x = readfile(os.path.join(data_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


## Basic setting

In [3]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

In [4]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'Inception3',
 'MNASNet',
 'MobileNetV2',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'video',
 'wide_resnet101_2',
 'wide_resnet50_2']

In [5]:
model = models.resnet50(pretrained=True).cuda()

for param in model.parameters():
    param.require_grad = False
    
fc = nn.Sequential(
    nn.Linear(512*4*4, 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 11)
)

model.classifier = fc

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/mj/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.7MB/s]


In [6]:
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 30

In [7]:
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader):
        X = data[0].cuda()
        Y = data[1].cuda()
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        
        #### What's Wrong
        train_pred = model(X) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
            
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

[001/030] 35.30 sec(s) Train Acc: 0.590209 Loss: 0.011330 | Val Acc: 0.670262 loss: 0.008041
[002/030] 35.53 sec(s) Train Acc: 0.746605 Loss: 0.006195 | Val Acc: 0.712828 loss: 0.006986
[003/030] 35.54 sec(s) Train Acc: 0.793432 Loss: 0.004957 | Val Acc: 0.711953 loss: 0.007075
[004/030] 35.28 sec(s) Train Acc: 0.820292 Loss: 0.004326 | Val Acc: 0.762682 loss: 0.006430
[005/030] 36.06 sec(s) Train Acc: 0.833773 Loss: 0.004024 | Val Acc: 0.693294 loss: 0.007669
[006/030] 35.51 sec(s) Train Acc: 0.831644 Loss: 0.004019 | Val Acc: 0.719825 loss: 0.007995
[007/030] 35.49 sec(s) Train Acc: 0.859315 Loss: 0.003257 | Val Acc: 0.753353 loss: 0.006905
[008/030] 35.58 sec(s) Train Acc: 0.860734 Loss: 0.003415 | Val Acc: 0.726531 loss: 0.007186
[009/030] 35.28 sec(s) Train Acc: 0.872491 Loss: 0.003049 | Val Acc: 0.731778 loss: 0.006893
[010/030] 35.31 sec(s) Train Acc: 0.903912 Loss: 0.002255 | Val Acc: 0.769679 loss: 0.006260
[011/030] 35.28 sec(s) Train Acc: 0.903912 Loss: 0.002282 | Val Acc: 0

In [ ]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [ ]:
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

In [8]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [10]:
# model_best.eval()
model.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [11]:
#將結果寫入 csv 檔
with open("../Output/predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))